In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!pip install cloud-tpu-client==0.10 -q
!pip install https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl -q

  ERROR: HTTP error 404 while getting https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl
  ERROR: Could not install requirement torch-xla==1.12.1 from https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl because of error 404 Client Error: Not Found for url: https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl
ERROR: Could not install requirement torch-xla==1.12.1 from https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl because of HTTP error 404 Client Error: Not Found for url: https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl for URL https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl


In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet
%pycat loss.py

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet


In [ ]:
%%writefile loss.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class VoxelNetLoss(nn.Module):
    def __init__(self, alpha, beta):
        super(VoxelNetLoss, self).__init__()
        self.smoothl1loss = nn.SmoothL1Loss(reduction=False)
        self.alpha = alpha
        self.beta = beta

    def forward(self, rm, psm, pos_equal_one, neg_equal_one, targets):

        p_pos = F.sigmoid(psm.permute(0,2,3,1))
        rm = rm.permute(0,2,3,1).contiguous()
        rm = rm.view(rm.size(0),rm.size(1),rm.size(2),-1,7)
        targets = targets.view(targets.size(0),targets.size(1),targets.size(2),-1,7)
        pos_equal_one_for_reg = pos_equal_one.unsqueeze(pos_equal_one.dim()).expand(-1,-1,-1,-1,7)

        rm_pos = rm * pos_equal_one_for_reg
        targets_pos = targets * pos_equal_one_for_reg

        cls_pos_loss = -pos_equal_one * torch.log(p_pos + 1e-6)
        cls_pos_loss = cls_pos_loss.sum() / (pos_equal_one.sum() + 1e-6)

        cls_neg_loss = -neg_equal_one * torch.log(1 - p_pos + 1e-6)
        cls_neg_loss = cls_neg_loss.sum() / (neg_equal_one.sum() + 1e-6)

        reg_loss = self.smoothl1loss(rm_pos, targets_pos)
        reg_loss = reg_loss / (pos_equal_one.sum() + 1e-6)
        conf_loss = self.alpha * cls_pos_loss + self.beta * cls_neg_loss
        return conf_loss, reg_loss

Overwriting loss.py


In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet

import torch
from torch import nn, optim

# import torch_xla
# import torch_xla.core.xla_model as xm

from model import VoxelNet
from loss import VoxelNetLoss
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

# model
net = VoxelNet()
try:
  device = xm.xla_device()
except: 
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("\n## Hardware type: {}\n".format(device.type))
net.to(device)

# training mode
net.train()

# initialization
print('Initializing weights...')
net.apply(weights_init)

# define optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# loss function
loss = VoxelNetLoss(alpha=1.5, beta=1)

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet

## Hardware type: cpu

Initializing weights...


In [4]:
%%writefile .gitignore
*/.ipynb_checkpoints/*
**.bin
**.txt
**.png
*.zip

Overwriting .gitignore
